In [2]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 93.1 MB/s eta 0:00:00


In [3]:
import pdfplumber

with pdfplumber.open("/content/ML.pdf") as pdf:
    text = ""
    for page in pdf.pages:
        text += page.extract_text()


In [4]:
import re
import json
qa_pairs = re.findall(r"(\d+ .*?\?)\n(.*?)(?=\n\d+|\Z)", text, re.DOTALL)

# Construire un dataset structuré
dataset = [{"prompt": q.strip(), "response": a.strip()} for q, a in qa_pairs]

# Sauvegarder en JSON
with open("qa_dataset.json", "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

# Afficher un aperçu
#for pair in dataset:
    #print(f"Question: {pair['prompt']}")
    #print(f"Réponse: {pair['response']}\n")

In [5]:
import json

# Charger le fichier JSON
with open('qa_dataset.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Vérifier le contenu
print(data[20:22])  # Affiche les 2 premiers éléments


[{'prompt': '29 Define non-negative matrix factorization. Give an example of its\napplication.\nMatrix factorization means factorizing a matrix into 2 or more matrices\nsuch that the product of these matrices approximates the actual matrix. This\ntechnique can greatly simplify the complex matrix operations and can be\nused to find the latent features in the given data, such as in a\nRecommendation system, where it could be used to find the similarities\nbetween two users.\nIn non-negative matrix factorization (NMF), a matrix is factorized into 2\nsub-matrices such that all the 3 matrices have no negative elements. Apart\nfrom Recommendation system, NMF can be applied to text mining, to\nquery from a set of documents.\nIn this technique, a document-term matrix is constructed from the input\ngroup of documents and then factorized into term-feature and feature-\ndocument matrices. It is also known as inverted indexing since it representsthe frequency of terms in each document and indexes 

In [6]:
import json

# Charger les données
with open('qa_dataset.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Créer un fichier texte pour le fine-tuning
with open('qa_finetuning.txt', 'w', encoding='utf-8') as file:
    for qa in data:
        file.write(f"Question: {qa['prompt']}\n")
        file.write(f"Answer: {qa['response']}\n\n")


In [7]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Charger le modèle et le tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Charger les données
def load_dataset(file_path, tokenizer, block_size=128):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )

def create_data_collator(tokenizer):
    return DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False  # Pour GPT, pas de Masked Language Modeling
    )

dataset = load_dataset("qa_finetuning.txt", tokenizer)
data_collator = create_data_collator(tokenizer)

# Configurer l'entraînement
training_args = TrainingArguments(
    output_dir="./gpt2_finetuned",
    overwrite_output_dir=True,
    num_train_epochs=4,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# Lancer l'entraînement
trainer.train()

# Sauvegarder le modèle finetuné
model.save_pretrained("./gpt2_finetuned")
tokenizer.save_pretrained("./gpt2_finetuned")


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss


('./gpt2_finetuned/tokenizer_config.json',
 './gpt2_finetuned/special_tokens_map.json',
 './gpt2_finetuned/vocab.json',
 './gpt2_finetuned/merges.txt',
 './gpt2_finetuned/added_tokens.json')

In [8]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Charger le modèle fine-tuné
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2_finetuned")
model = GPT2LMHeadModel.from_pretrained("./gpt2_finetuned")

# Ajouter le token de padding
tokenizer.pad_token = tokenizer.eos_token

# Tester avec une question
input_text = "Question: What is backpropagation in neural networks? Provide a concise explanation."
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Créer l'attention mask (1 pour les tokens non-padding, 0 pour les tokens padding)
attention_mask = (input_ids != tokenizer.pad_token_id).long()

# Générer une réponse
output = model.generate(
    input_ids,
    attention_mask=attention_mask,  # Passer l'attention mask ici
    max_length=100,  # Augmenter max_length pour plus de contenu
    num_return_sequences=1,
    temperature=1.0,  # Augmenter la température pour plus de diversité
    top_p=0.9,
    no_repeat_ngram_size=2  # Empêcher la répétition des bigrammes
)

# Décoder et afficher la réponse
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: What is backpropagation in neural networks? Provide a concise explanation.
Answer: Backpropagging is a technique used to train a neural network to perform a task. Backward training is when the training dataset is not completely accurate, but the model is still performing the task correctly. It is used for training a model to predict the outcome of the given task, such as the number of times the person is present in the present time. The goal is to minimize the error in predicting


In [39]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Charger le modèle et le tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2_finetuned")
model = GPT2LMHeadModel.from_pretrained("./gpt2_finetuned")

# Exemple de texte pour l'évaluation
input_text = "Question:What is backpropagation?"

# Tokenisation du texte d'entrée
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Calcul de la perplexité
with torch.no_grad():
    outputs = model(input_ids, labels=input_ids)
    loss = outputs.loss
    perplexity = torch.exp(loss)

print(f"Perplexity: {perplexity.item()}")


Perplexity: 19.07032585144043
